In [1]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib

In [ ]:
data_dir = '.\dataset\moc'

In [ ]:
data_root = pathlib.Path(data_dir)

In [ ]:
data_root

In [ ]:
for item in data_root.iterdir():
    print(item)

In [ ]:
all_image_paths = list(data_root.glob('*/*'))

In [ ]:
image_count = len(all_image_paths)
image_count #取出所有图片

In [ ]:
all_image_paths[-3:]

In [ ]:
# 对所有图片的路径进行乱序
import random
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

In [ ]:
all_image_paths[-3:]

In [ ]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

In [ ]:
color_label_names = set(name.split('_')[0] for name in label_names)
color_label_names#提取颜色

In [ ]:
item_label_names = set(name.split('_')[1] for name in label_names)
item_label_names#提取物品

In [ ]:
color_label_to_index = dict((name, index) for index,name in enumerate(color_label_names))
color_label_to_index

In [ ]:
item_label_to_index = dict((name, index) for index,name in enumerate(item_label_names))
item_label_to_index

In [ ]:
import IPython.display as display
index_to_color =dict( (v, k) for k, v in color_label_to_index.items())

index_to_color #知道编号可获取实际名称

In [ ]:
import IPython.display as display
index_to_item =dict( (v, k) for k, v in item_label_to_index.items())

index_to_item #知道编号可获取实际名称

In [ ]:
#对label进行编码

In [ ]:
all_image_labels = [pathlib.Path(path).parent.name for path in all_image_paths]
all_image_labels[:5]

In [ ]:
color_labels = [color_label_to_index[label.split('_')[0]] for label in all_image_labels]
color_labels[:5]

In [ ]:
item_labels = [item_label_to_index[label.split('_')[1]] for label in all_image_labels]
item_labels[:5]

In [ ]:
import IPython.display as display

In [ ]:
for n in range(3):
    image_index = random.choice(range(len(all_image_paths)))
    display.display(display.Image(all_image_paths[image_index],width=100, height=100))
    print(all_image_labels[image_index])
    print()

In [ ]:
#加载和格式化图片

In [ ]:
img_path = all_image_paths[0]
img_path

In [ ]:
img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")

In [ ]:
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

In [ ]:
img_tensor = tf.cast(img_tensor, tf.float32)
img_tensor = tf.image.resize(img_tensor, [224,224])
img_final = img_tensor/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())

In [ ]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path) #读取图片
    image = tf.image.decode_jpeg(image, channels=3) #解码
    image = tf.image.resize(image, [244,244]) #重新定义大小为244*244
    image = tf.cast(image, tf.float32) #改变数据类型
    image = image / 255.0 #归一化到0-1
    image = 2*image-1
    return image

In [ ]:
import matplotlib.pyplot as plt

image_path = all_image_paths[0]
label = all_image_labels[0]

plt.imshow((load_and_preprocess_image(img_path)+1)/2) #由-1到1 变为0到1
plt.grid(False)
plt.xlabel(label)
print()

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

In [ ]:
label_ds = tf.data.Dataset.from_tensor_slices((color_labels, item_labels))

In [ ]:
for ele in label_ds.take(3):
    print(ele[0].numpy(), ele[1].numpy())

In [ ]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [ ]:
image_label_ds

In [ ]:
#划分test数据集
test_count = int(image_count*0.2)
train_count = image_count - test_count

In [ ]:
train_data = image_label_ds.skip(test_count)

test_data = image_label_ds.take(test_count)

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_data = train_data.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)
train_data

In [ ]:
test_data = test_data.batch(BATCH_SIZE)

In [ ]:
# 11.2 建立模型

In [ ]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False
                                              )

In [ ]:
inputs = tf.keras.Input(shape=(244, 244, 3))

In [ ]:
x = mobile_net(inputs)

In [ ]:
x.get_shape() #输出变成四维，所以需要添加GlobalAveragePooling2D层

In [ ]:
x =tf.keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
x.get_shape() #现在变成[None, 1280]形式

In [ ]:
x1 = tf.keras.layers.Dense(1024, activation='relu')(x)
out_color = tf.keras.layers.Dense(len(color_label_names), activation='softmax', 
                                  name='out_color')(x1)

In [ ]:
x2 = tf.keras.layers.Dense(1024, activation='relu')(x)
out_item = tf.keras.layers.Dense(len(item_label_names), activation='softmax',
                                name='out_item')(x2)

In [ ]:
model = tf.keras.Model(inputs=inputs,
                      outputs=[out_color,out_item])

In [ ]:
model.summary()

In [ ]:
#11.3 编译 配置

In [ ]:
model.compile(optimizer='adam',
              loss={'out_color':'sparse_categorical_crossentropy',
                    'out_item':'sparse_categorical_crossentropy'},#两个输出，loss赋值为字典,两个使用相同的loss可以只写一个
              metrics=['acc']
             )

In [ ]:
train_steps = train_count//BATCH_SIZE
test_steps = test_count//BATCH_SIZE

In [ ]:
history = model.fit(train_data,
          epochs=15,
          steps_per_epoch=train_steps,
          validation_data=test_data,
          validation_steps=test_steps)

In [ ]:
#11.4 评价模型

In [ ]:
model.evaluate(test_data,verbose=1) # verbose为0不显示提示

In [ ]:
my_image = load_and_preprocess_image(r'C:\Users\i80980\Downloads\863b601ff3d38ee259665b78dfc13d58.jpg')

In [ ]:
my_image.shape

In [ ]:
#拓展成四维图片
my_image = np.expand_dims(my_image, 0) #或者使用tf.expand_dims

In [ ]:
my_image.shape

In [ ]:
#使用模型对单张图片进行预测 #以下为预测的概率值
pre = model.predict(my_image)
pre

In [ ]:
#model(my_image,training=False) #也可以直接调用model进行预测

In [ ]:
pre[0] #对第一个color的预测

In [ ]:
pre_color = index_to_color.get(np.argmax(pre[0][0]))

In [ ]:
pre_color

In [ ]:
pre_item = index_to_item.get(np.argmax(pre[1][0]))

In [ ]:
pre_item

In [ ]:
pred = pre_color + '_' + pre_item
pred

In [ ]:
plt.imshow((my_image[0] + 1)/2)
plt.xlabel(pred)

In [ ]:
model(my_image)

In [ ]:
json_config = model.to_json()

In [ ]:
json_config

In [ ]:
#重建模型

In [ ]:
reinitialized_model = tf.keras.models.model_from_json(json_config)

In [ ]:
reinitialized_model.summary()

In [ ]:
reinitialized_model.compile(optimizer='adam',
              loss={'out_color':'sparse_categorical_crossentropy',
                    'out_item':'sparse_categorical_crossentropy'},#两个输出，loss赋值为字典,两个使用相同的loss可以只写一个
              metrics=['acc']
             )

In [ ]:
reinitialized_model.evaluate(test_data,verbose=1)

In [ ]:
#仅保存权重

In [ ]:
weighs = model.get_weights()

In [ ]:
weighs

In [ ]:
reinitialized_model.set_weights(weighs)

In [ ]:
reinitialized_model.evaluate(test_data,verbose=1)

In [ ]:
#将权重保存到磁盘上

In [ ]:
model.save_weights('less_weight.h5')

In [ ]:
#从磁盘上加载权重
reinitialized_model.load_weights('less_weight.h5')

In [ ]:
#使用上面两个方法 并不等同于第一种方法，但仍然没有保存优化器的配置    

In [ ]:
checkpoint_path = 'training_cp/cp.ckpt' #保存检查点的路径

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True) #设置回调函数 只保存权重